# PyPortifolio — Solicitação de melhorias (v2.4)

Quero que você analise profundamente o notebook **Otimizacao_de_Portfolio_v2_3.ipynb** e proponha/implemente melhorias.  
O objetivo é evoluir o projeto para uma versão mais robusta, com fallback de fontes de dados, restrições reais de carteira, Black-Litterman e backtesting out-of-sample.

## Caminhos do projeto
- Notebook atual: `Otimizacao_de_Portfolio_v2_3.ipynb`
- Arquivo Excel (retornos/pesos) está no projeto em:
  - `1 - Dados\1 - Rentabilidade atual\Rendimentos_Mensais_Ativos_v7.0.xlsx`

## O que eu quero que você faça (obrigatório)

### 1) Diagnóstico profundo + plano priorizado
- Leia o notebook e identifique:
  - arquitetura atual, fluxos, variáveis globais, funções-chave
  - pontos frágeis (ex.: annualização, PSD da cov, histórico curto, ativos “flat”, tratamento de missing)
  - gargalos de usabilidade e manutenção
- Entregue um plano de melhorias priorizado (alto impacto primeiro), justificando.

### 2) Notebook revisado, organizado e comentado
- Entregar uma nova versão organizada e bem comentada: **v2.4**
- Manter a estrutura por blocos/etapas, mas:
  - centralizar configurações no topo (paths, flags, MIN_OBS, MIN_OVERLAP, VOL_FLOOR, RF, etc.)
  - agrupar funções utilitárias (loading, estatísticas, otimização, plots)
  - inserir logs/prints padronizados e mensagens claras de diagnóstico

---

## Melhorias específicas por limitação (v2.3)

### 3) Implementar yfinance como fonte secundária (hierarquia de fontes)
Atualmente a v2.3 usa somente Excel. Quero implementar a hierarquia abaixo no CÓDIGO (comportamento determinístico):

1) **Se o usuário escolher usar Excel** → usar retornos/pesos do Excel como fonte primária  
2) **Se NÃO usar Excel**, mas houver carteira/tickers informados/derivados → baixar preços com `yfinance` e gerar retornos  
3) **Se não houver carteira**, usar `DEFAULT_UNIVERSE` explícito (lista fixa em config) e baixar com `yfinance`  
4) Se nada existir → erro claro e instrução de como resolver (sem inventar ativos)

O notebook deve perguntar:
- “Carregar dados do Excel (retornos/pesos)? (s/n) [s]”
Se “não”, seguir fluxo yfinance/default sem quebrar.

Regras:
- Usar `Adj Close` quando aplicável.
- Converter preços em retornos na mesma frequência usada na otimização (preferência: mensal).
- Registrar falhas de tickers e aplicar fallback (ticker alternativo) quando houver; senão excluir do universo e reportar.

---

### 4) Restrições por classe (alocação realista)
Implementar restrições opcionais por classe (exemplos):
- `cripto` max 15%
- `acoes_br` max 35%
- `rf` min 20%
- `global` max 40%
- limite por ativo (ex.: max 20%)

Como obter classes:
- Preferência: ler do Excel (se existir coluna/aba com classe) ou manter um dicionário `MAPA_CLASSE_ATIVO` configurável.
- Se não houver classe de um ativo: marcar como “SemClasse” e incluir em um grupo padrão com limites amplos (mas reportar).

Implementação:
- Otimização Markowitz deve aceitar constraints por classe (soma dos pesos do grupo) + bounds por ativo.
- Incluir no relatório final: pesos por classe antes/depois e verificação das constraints.

---

### 5) Implementar Black-Litterman (além de Markowitz)
Adicionar um modo de otimização Black-Litterman e comparação com Markowitz.

Requisitos mínimos:
- Calcular retornos de equilíbrio (implied returns) a partir de:
  - `w_mkt` (pode ser “carteira atual” se pesos carregados, ou equal-weight/default)
  - `Sigma` e parâmetro de aversão a risco `delta`
- Permitir “views” opcionais do usuário:
  - views absolutas (ex.: “Ativo X terá 12% a.a.”)
  - views relativas (ex.: “Ativo A retornará 2% acima do B”)
- Se não houver views: usar BL apenas como suavização robusta (prior do mercado).
- Entregar: carteira BL (mín variância / máx Sharpe) e comparar com Markowitz:
  - retorno esperado, vol, Sharpe, pesos, concentração (HHI), contribuição de risco.

---

### 6) Melhorar fuzzy matching e auditoria de mapeamento
O fuzzy atual é simples. Melhorar para reduzir erros e aumentar transparência.

Requisitos:
- Implementar normalização de nomes (acentos, caixa, caracteres especiais).
- Usar algoritmo robusto (ex.: token_set_ratio / partial_ratio; se possível, usar `rapidfuzz`).
- Gerar uma tabela “mapeamento” com:
  - ativo_original → ativo_mapeado → score → método (exact/contains/fuzzy) → status (OK/REVISAR)
- Definir limiar:
  - `score >= 90`: aceita automaticamente
  - `70 <= score < 90`: aceita mas marca como “REVISAR”
  - `score < 70`: não mapear (excluir) e listar no relatório final
- Nunca inventar mapeamento.

---

### 7) Implementar backtesting (out-of-sample) e relatório
A v2.3 é in-sample. Implementar backtesting mínimo:

Opção A (simples e suficiente):
- Split temporal: `train` (ex.: 70%) e `test` (30%)
- Otimiza em `train`, aplica pesos fixos em `test`
- Reporta métricas no test: retorno anualizado, vol, Sharpe, max drawdown

Opção B (melhor):
- Walk-forward / janela rolante (ex.: 24m treina, 1m aplica; rebalance mensal/trimestral)
- Incluir turnover e custo de transação opcional (bps) para realismo

Gráficos:
- Curva de patrimônio (cumulative return) da carteira atual vs GMV vs Máx Sharpe vs BL
- Drawdown
- Rolling vol / rolling Sharpe (opcional)

---

## Correções e melhorias baseadas na figura da fronteira (v2.3)
No gráfico atual, notei possíveis sintomas:
- GMV com vol ~0% (provável série “flat”/baixa variância ou cov mal condicionada)
- Sharpe muito negativo para quase tudo (RF alto vs retornos do universo / inconsistência de frequência)
Quero que você:
1) adicione filtros de qualidade (`MIN_OBS`, `MIN_OVERLAP`)
2) aplique `VOL_FLOOR` (piso de volatilidade) configurável para ativos muito “suaves”
3) garanta matriz PSD (near-PSD) e estabilidade numérica
4) revise anualização e retorno esperado (preferência: geométrico/log) e alinhe com RF

---

## Visualizações Plotly (obrigatório)
No gráfico principal:
- Nuvem Monte Carlo + linha da fronteira eficiente (QP por retorno-alvo)
- Marcar:
  - GMV
  - Máx Sharpe
  - Carteira Atual (estrela com cor exclusiva)
  - (novo) BL (símbolo distinto)
- Hover:
  - retorno anual, vol anual, Sharpe
  - top pesos (top 10)
- Heatmap de correlação
- Barras de pesos (top N), incluindo por classe

---

## Entregáveis
1) Lista de melhorias priorizadas + justificativas
2) Notebook v2.4 revisado, organizado e comentado
3) (Opcional recomendado) Extrair funções para um módulo `.py` (ex.: `portfolio_core.py`) e manter notebook como “orquestrador”
4) Checklist de validação (rodar do zero; com/sem Excel; com/sem carteira; com yfinance)
